# AWS vs MODIS Albedo Scatterplot Matrix Generator

This notebook recreates the exact 3×3 AWS vs MODIS albedo correlation plots from the multi-glacier comparative analysis. It includes the complete data pipeline from raw CSV files to publication-ready visualizations.

## Features
- **Complete Data Pipeline**: Loads data from all 3 glaciers with proper format handling
- **Intelligent Pixel Selection**: Uses distance to AWS stations + glacier fraction weighting  
- **Outlier Filtering**: Applies 2.5σ threshold for robust statistics
- **Comprehensive Statistics**: Calculates R, R², RMSE, MAE, Bias for each method
- **Publication-Ready Plots**: Exact 3×3 matrix with proper colors and styling
- **Interactive Analysis**: Easy to modify parameters and explore results

## Glaciers Analyzed
- **Athabasca** (Canadian Rockies): Blue color, all 2 pixels used
- **Haig** (Canadian Rockies): Orange color, best pixel selected
- **Coropuna** (Peruvian Andes): Green color, best pixel selected

## MODIS Methods
- **MCD43A3**: Daily BRDF-adjusted reflectance
- **MOD09GA**: Daily surface reflectance
- **MOD10A1**: Daily snow cover


## 1. Setup and Configuration

In [ ]:
#!/usr/bin/env python3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
from pathlib import Path
from typing import Dict, List, Any, Optional, Tuple
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set up logging for notebook
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✓ All libraries imported successfully")

In [ ]:
# Configuration - Modify these paths to match your system
CONFIG = {
    'data_paths': {
        'athabasca': {
            'modis': "D:/Documents/Projects/athabasca_analysis/data/csv/Athabasca_Terra_Aqua_MultiProduct_2014-01-01_to_2021-01-01.csv",
            'aws': "D:/Documents/Projects/athabasca_analysis/data/csv/iceAWS_Atha_albedo_daily_20152020_filled_clean.csv"
        },
        'haig': {
            'modis': "D:/Documents/Projects/Haig_analysis/data/csv/Haig_MODIS_Pixel_Analysis_MultiProduct_2002_to_2016_fraction.csv",
            'aws': "D:/Documents/Projects/Haig_analysis/data/csv/HaigAWS_daily_2002_2015_gapfilled.csv"
        },
        'coropuna': {
            'modis': "D:/Documents/Projects/Coropuna_glacier/data/csv/coropuna_glacier_2014-01-01_to_2025-01-01.csv",
            'aws': "D:/Documents/Projects/Coropuna_glacier/data/csv/COROPUNA_simple.csv"
        }
    },
    'aws_stations': {
        'athabasca': {'lat': 52.1949, 'lon': -117.2431, 'name': 'Athabasca AWS'},
        'haig': {'lat': 50.7186, 'lon': -115.3433, 'name': 'Haig AWS'},
        'coropuna': {'lat': -15.5181, 'lon': -72.6617, 'name': 'Coropuna AWS'}
    },
    'colors': {
        'athabasca': '#1f77b4',  # Blue
        'haig': '#ff7f0e',       # Orange  
        'coropuna': '#2ca02c'    # Green
    },
    'methods': ['MCD43A3', 'MOD09GA', 'MOD10A1'],
    'outlier_threshold': 2.5,
    'quality_filters': {
        'min_glacier_fraction': 0.1,
        'min_observations': 10
    },
    'visualization': {
        'figsize': (15, 12),
        'dpi': 300,
        'style': 'seaborn-v0_8'
    }
}

print("✓ Configuration loaded")
print(f"Glaciers: {list(CONFIG['data_paths'].keys())}")
print(f"Methods: {CONFIG['methods']}")
print(f"Outlier threshold: {CONFIG['outlier_threshold']}σ")

## 2. Data Loading Classes

In [ ]:
class DataLoader:
    """Handles loading and preprocessing of MODIS and AWS data for all glaciers."""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        
    def load_glacier_data(self, glacier_id: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Load MODIS and AWS data for a specific glacier."""
        logger.info(f"Loading data for {glacier_id} glacier...")
        
        paths = self.config['data_paths'][glacier_id]
        
        # Load MODIS data
        modis_data = self._load_modis_data(paths['modis'], glacier_id)
        
        # Load AWS data
        aws_data = self._load_aws_data(paths['aws'], glacier_id)
        
        logger.info(f"Loaded {len(modis_data):,} MODIS and {len(aws_data):,} AWS records for {glacier_id}")
        
        return modis_data, aws_data
    
    def _load_modis_data(self, file_path: str, glacier_id: str) -> pd.DataFrame:
        """Load MODIS data with glacier-specific parsing."""
        if not Path(file_path).exists():
            raise FileNotFoundError(f"MODIS data file not found: {file_path}")
        
        logger.info(f"Loading MODIS data from: {file_path}")
        data = pd.read_csv(file_path)
        
        # Convert date to datetime
        data['date'] = pd.to_datetime(data['date'])
        
        # Glacier-specific processing
        if glacier_id == 'coropuna':
            # Coropuna has method column - already in long format
            if 'method' in data.columns and 'albedo' in data.columns:
                logger.info("Coropuna data is in long format")
                return data
        
        # For other glaciers, check if conversion to long format is needed
        if 'method' not in data.columns:
            logger.info(f"Converting {glacier_id} data to long format")
            data = self._convert_to_long_format(data, glacier_id)
        
        return data
    
    def _convert_to_long_format(self, data: pd.DataFrame, glacier_id: str) -> pd.DataFrame:
        """Convert wide format MODIS data to long format."""
        long_format_rows = []
        
        # Define method mappings based on available columns
        method_columns = {}
        for col in data.columns:
            if 'MOD09GA' in col and 'albedo' in col:
                method_columns['MOD09GA'] = col
            elif 'MOD10A1' in col and 'albedo' in col:
                method_columns['MOD10A1'] = col
            elif 'MCD43A3' in col and 'albedo' in col:
                method_columns['MCD43A3'] = col
            elif col in ['MOD09GA', 'MOD10A1', 'MCD43A3']:
                method_columns[col] = col
        
        for method, col_name in method_columns.items():
            if col_name not in data.columns:
                continue
                
            # Extract data for this method
            method_data = data[data[col_name].notna()][['pixel_id', 'date', col_name]].copy()
            
            if len(method_data) > 0:
                method_data['method'] = method
                method_data['albedo'] = method_data[col_name]
                method_data = method_data.drop(columns=[col_name])
                
                # Add spatial coordinates if available
                for coord_col in ['longitude', 'latitude']:
                    if coord_col in data.columns:
                        method_data[coord_col] = data.loc[method_data.index, coord_col]
                
                # Add glacier fraction if available
                glacier_frac_cols = [c for c in data.columns if 'glacier_fraction' in c.lower()]
                if glacier_frac_cols:
                    method_data['glacier_fraction'] = data.loc[method_data.index, glacier_frac_cols[0]]
                
                long_format_rows.append(method_data)
        
        if long_format_rows:
            return pd.concat(long_format_rows, ignore_index=True)
        else:
            logger.error(f"No valid method data found for {glacier_id}")
            return pd.DataFrame()
    
    def _load_aws_data(self, file_path: str, glacier_id: str) -> pd.DataFrame:
        """Load AWS data with glacier-specific parsing."""
        if not Path(file_path).exists():
            raise FileNotFoundError(f"AWS data file not found: {file_path}")
        
        logger.info(f"Loading AWS data from: {file_path}")
        
        if glacier_id == 'haig':
            # Haig has special format: semicolon separated, European decimal
            aws_data = pd.read_csv(file_path, sep=';', skiprows=6, decimal=',')
            aws_data.columns = aws_data.columns.str.strip()
            
            # Process Year and Day columns to create datetime
            aws_data = aws_data.dropna(subset=['Year', 'Day'])
            aws_data['Year'] = aws_data['Year'].astype(int)
            aws_data['Day'] = aws_data['Day'].astype(int)
            
            # Convert Day of Year to datetime
            aws_data['date'] = pd.to_datetime(
                aws_data['Year'].astype(str) + '-01-01'
            ) + pd.to_timedelta(aws_data['Day'] - 1, unit='D')
            
            # Find albedo column
            albedo_cols = [col for col in aws_data.columns if 'albedo' in col.lower()]
            if albedo_cols:
                albedo_col = albedo_cols[0]
                aws_data['Albedo'] = pd.to_numeric(aws_data[albedo_col], errors='coerce')
            else:
                raise ValueError(f"No albedo column found in Haig AWS data")
                
        elif glacier_id == 'coropuna':
            # Coropuna format: Timestamp, Albedo
            aws_data = pd.read_csv(file_path)
            aws_data['date'] = pd.to_datetime(aws_data['Timestamp'])
            
        elif glacier_id == 'athabasca':
            # Athabasca format: Time, Albedo
            aws_data = pd.read_csv(file_path)
            aws_data['date'] = pd.to_datetime(aws_data['Time'])
        
        # Clean and validate data
        aws_data = aws_data[['date', 'Albedo']].copy()
        aws_data = aws_data.dropna(subset=['Albedo'])
        aws_data = aws_data[aws_data['Albedo'] > 0]  # Remove invalid albedo values
        aws_data = aws_data.drop_duplicates().sort_values('date').reset_index(drop=True)
        
        return aws_data

print("✓ DataLoader class defined")

## 3. Pixel Selection Algorithm

In [ ]:
class PixelSelector:
    """Implements intelligent pixel selection based on distance to AWS stations."""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        
    def select_best_pixels(self, modis_data: pd.DataFrame, glacier_id: str) -> pd.DataFrame:
        """Select best pixels for analysis based on AWS distance and glacier fraction."""
        logger.info(f"Applying pixel selection for {glacier_id}...")
        
        # Get AWS station coordinates
        aws_station = self.config['aws_stations'][glacier_id]
        aws_lat, aws_lon = aws_station['lat'], aws_station['lon']
        
        # Get available pixels with their quality metrics
        pixel_summary = modis_data.groupby('pixel_id').agg({
            'glacier_fraction': 'mean',
            'albedo': 'count',
            'latitude': 'first',
            'longitude': 'first'
        }).reset_index()
        
        pixel_summary.columns = ['pixel_id', 'avg_glacier_fraction', 'n_observations', 'latitude', 'longitude']
        
        # Apply quality filters
        quality_filters = self.config['quality_filters']
        quality_pixels = pixel_summary[
            (pixel_summary['avg_glacier_fraction'] > quality_filters['min_glacier_fraction']) & 
            (pixel_summary['n_observations'] > quality_filters['min_observations'])
        ].copy()
        
        if len(quality_pixels) == 0:
            logger.warning(f"No quality pixels found for {glacier_id}, using all data")
            return modis_data
        
        # Calculate distance to AWS station using Haversine formula
        quality_pixels['distance_to_aws'] = self._haversine_distance(
            quality_pixels['latitude'], quality_pixels['longitude'], aws_lat, aws_lon
        )
        
        # For Athabasca (small dataset), use all pixels
        if glacier_id == 'athabasca':
            selected_pixel_ids = quality_pixels['pixel_id'].tolist()
            logger.info(f"Using all {len(selected_pixel_ids)} pixels for {glacier_id} (small dataset)")
        else:
            # Sort by glacier fraction (descending) then distance (ascending)
            quality_pixels = quality_pixels.sort_values([
                'avg_glacier_fraction', 'distance_to_aws'
            ], ascending=[False, True])
            
            # Select the best performing pixel
            selected_pixels = quality_pixels.head(1)
            selected_pixel_ids = selected_pixels['pixel_id'].tolist()
            
            logger.info(f"Selected {len(selected_pixel_ids)} best pixel(s) for {glacier_id}")
            for _, pixel in selected_pixels.iterrows():
                logger.info(f"  Pixel {pixel['pixel_id']}: "
                           f"glacier_fraction={pixel['avg_glacier_fraction']:.3f}, "
                           f"distance={pixel['distance_to_aws']:.2f}km, "
                           f"observations={pixel['n_observations']}")
        
        # Filter MODIS data to selected pixels
        filtered_data = modis_data[modis_data['pixel_id'].isin(selected_pixel_ids)].copy()
        logger.info(f"Filtered MODIS data from {len(modis_data)} to {len(filtered_data)} observations")
        
        return filtered_data
    
    def _haversine_distance(self, lat1, lon1, lat2, lon2):
        """Calculate distance using Haversine formula."""
        R = 6371  # Earth's radius in km
        lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
        c = 2 * np.arcsin(np.sqrt(a))
        return R * c

print("✓ PixelSelector class defined")

## 4. Data Processing and Statistics

In [ ]:
class DataProcessor:
    """Handles AWS-MODIS data merging and statistical processing."""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        
    def merge_and_process(self, modis_data: pd.DataFrame, aws_data: pd.DataFrame, 
                         glacier_id: str) -> pd.DataFrame:
        """Merge AWS and MODIS data and calculate statistics for each method."""
        logger.info(f"Merging and processing data for {glacier_id}...")
        
        results = []
        methods = self.config['methods']
        
        for method in methods:
            # Filter MODIS data for this method
            method_data = modis_data[modis_data['method'] == method].copy()
            
            if len(method_data) == 0:
                logger.warning(f"No {method} data found for {glacier_id}")
                continue
            
            # Merge with AWS data on date
            merged = method_data.merge(aws_data, on='date', how='inner')
            
            if len(merged) < 3:  # Need minimum data points
                logger.warning(f"Insufficient {method} data for {glacier_id}: {len(merged)} points")
                continue
            
            # Apply outlier filtering
            aws_clean, modis_clean = self._apply_outlier_filtering(
                merged['Albedo'].values, merged['albedo'].values
            )
            
            if len(aws_clean) < 3:
                logger.warning(f"Insufficient {method} data after outlier filtering for {glacier_id}")
                continue
            
            # Calculate statistics
            stats = self._calculate_statistics(aws_clean, modis_clean)
            
            results.append({
                'glacier_id': glacier_id,
                'method': method,
                'aws_values': aws_clean,
                'modis_values': modis_clean,
                **stats
            })
            
            logger.info(f"Processed {method} for {glacier_id}: "
                       f"{len(aws_clean)} samples, r={stats['correlation']:.3f}")
        
        return pd.DataFrame(results)
    
    def _apply_outlier_filtering(self, aws_vals: np.ndarray, modis_vals: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """Apply 2.5σ outlier filtering to AWS-MODIS pairs."""
        if len(aws_vals) < 3:
            return aws_vals, modis_vals
        
        # Calculate residuals
        residuals = modis_vals - aws_vals
        mean_residual = np.mean(residuals)
        std_residual = np.std(residuals)
        
        # Apply threshold
        threshold = self.config['outlier_threshold'] * std_residual
        mask = np.abs(residuals - mean_residual) <= threshold
        
        return aws_vals[mask], modis_vals[mask]
    
    def _calculate_statistics(self, aws_vals: np.ndarray, modis_vals: np.ndarray) -> Dict[str, float]:
        """Calculate comprehensive statistics between AWS and MODIS values."""
        if len(aws_vals) == 0:
            return {
                'correlation': np.nan, 'r_squared': np.nan, 'rmse': np.nan,
                'mae': np.nan, 'bias': np.nan, 'n_samples': 0, 'p_value': 1.0
            }
        
        # Basic statistics
        correlation = np.corrcoef(aws_vals, modis_vals)[0, 1] if len(aws_vals) > 1 else np.nan
        r_squared = correlation**2 if not np.isnan(correlation) else np.nan
        
        # Error metrics
        residuals = modis_vals - aws_vals
        rmse = np.sqrt(np.mean(residuals**2))
        mae = np.mean(np.abs(residuals))
        bias = np.mean(residuals)
        
        # Statistical significance
        if len(aws_vals) > 2 and not np.isnan(correlation):
            t_stat = correlation * np.sqrt((len(aws_vals) - 2) / (1 - correlation**2))
            p_value = 2 * (1 - stats.t.cdf(abs(t_stat), len(aws_vals) - 2))
        else:
            p_value = 1.0
        
        return {
            'correlation': correlation if not np.isnan(correlation) else 0.0,
            'r_squared': r_squared if not np.isnan(r_squared) else 0.0,
            'rmse': rmse,
            'mae': mae,
            'bias': bias,
            'n_samples': len(aws_vals),
            'p_value': p_value
        }

print("✓ DataProcessor class defined")

## 5. Visualization Class

In [ ]:
class ScatterplotVisualizer:
    """Creates the 3×3 AWS vs MODIS scatterplot matrix."""
    
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        
    def create_scatterplot_matrix(self, processed_data: List[pd.DataFrame], 
                                output_path: Optional[str] = None) -> plt.Figure:
        """Create the 3×3 scatterplot matrix."""
        logger.info("Creating AWS vs MODIS scatterplot matrix...")
        
        # Set matplotlib style
        try:
            plt.style.use(self.config['visualization']['style'])
        except:
            logger.warning("Could not set plotting style, using default")
        
        # Create figure and subplots
        fig, axes = plt.subplots(3, 3, figsize=self.config['visualization']['figsize'])
        
        # Define layout: glaciers (rows) × methods (columns)
        glaciers = ['athabasca', 'coropuna', 'haig']  # Match your image order
        methods = self.config['methods']
        
        # Create title with pixel selection info
        subtitle = "Selected Best Pixels: 2/1/1 (Closest to AWS Stations)"
        fig.suptitle(f'AWS vs MODIS Albedo Correlations\n{subtitle}', 
                    fontsize=16, fontweight='bold')
        
        # Create scatterplots for each glacier-method combination
        for i, glacier_id in enumerate(glaciers):
            # Get data for this glacier
            glacier_data = next((df for df in processed_data if df['glacier_id'].iloc[0] == glacier_id), pd.DataFrame())
            
            for j, method in enumerate(methods):
                ax = axes[i, j]
                
                if not glacier_data.empty:
                    # Get data for this method
                    method_data = glacier_data[glacier_data['method'] == method]
                    
                    if not method_data.empty and len(method_data) > 0:
                        row = method_data.iloc[0]
                        aws_vals = row['aws_values']
                        modis_vals = row['modis_values']
                        
                        # Create scatterplot
                        self._create_single_scatterplot(ax, aws_vals, modis_vals, 
                                                      glacier_id, method, row)
                    else:
                        self._create_empty_plot(ax, glacier_id, method)
                else:
                    self._create_empty_plot(ax, glacier_id, method)
                
                # Set subplot title and labels
                ax.set_title(f'{glacier_id.title()} - {method}', 
                           fontsize=12, fontweight='bold')
                ax.set_xlabel('AWS Albedo', fontsize=10)
                ax.set_ylabel('MODIS Albedo', fontsize=10)
                ax.grid(True, alpha=0.3)
                ax.set_aspect('equal', adjustable='box')
                ax.set_xlim(0, 1)
                ax.set_ylim(0, 1)
        
        plt.tight_layout()
        plt.subplots_adjust(wspace=0.05)  # Make plots closer horizontally
        
        # Save figure if path provided
        if output_path:
            fig.savefig(output_path, dpi=self.config['visualization']['dpi'], 
                       bbox_inches='tight', facecolor='white')
            logger.info(f"Scatterplot matrix saved to: {output_path}")
        
        return fig
    
    def _create_single_scatterplot(self, ax, aws_vals: np.ndarray, modis_vals: np.ndarray,
                                 glacier_id: str, method: str, stats_row: pd.Series):
        """Create a single scatterplot with statistics."""
        # Get glacier color
        color = self.config['colors'][glacier_id]
        
        # Create scatter plot
        ax.scatter(aws_vals, modis_vals, alpha=0.6, s=20, color=color)
        
        # Add 1:1 reference line
        ax.plot([0, 1], [0, 1], 'k--', alpha=0.7, linewidth=1, label='1:1 line')
        
        # Add trend line
        if len(aws_vals) > 1:
            z = np.polyfit(aws_vals, modis_vals, 1)
            p = np.poly1d(z)
            ax.plot(aws_vals, p(aws_vals), 'r-', alpha=0.8, linewidth=1.5)
        
        # Add statistics text
        stats_text = (f'R = {stats_row["correlation"]:.3f}\n'
                     f'R² = {stats_row["r_squared"]:.3f}\n'
                     f'RMSE = {stats_row["rmse"]:.3f}\n'
                     f'MAE = {stats_row["mae"]:.3f}\n'
                     f'Bias = {stats_row["bias"]:.3f}\n'
                     f'n = {stats_row["n_samples"]}')
        
        ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, 
               verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.9),
               fontsize=8)
    
    def _create_empty_plot(self, ax, glacier_id: str, method: str):
        """Create empty plot for missing data."""
        ax.text(0.5, 0.5, 'No data available', transform=ax.transAxes,
               ha='center', va='center', fontsize=10, style='italic')
        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)

print("✓ ScatterplotVisualizer class defined")

## 6. Data Processing Workflow

Now let's run the complete analysis pipeline for all three glaciers:

In [ ]:
# Initialize all processing components
data_loader = DataLoader(CONFIG)
pixel_selector = PixelSelector(CONFIG)
data_processor = DataProcessor(CONFIG)
visualizer = ScatterplotVisualizer(CONFIG)

print("✓ All processing components initialized")
print("Ready to process glacier data...")

In [ ]:
# Process each glacier
all_processed_data = []
processing_summary = []

for glacier_id in ['athabasca', 'haig', 'coropuna']:
    try:
        print(f"\n{'='*50}")
        print(f"Processing {glacier_id.upper()} Glacier")
        print(f"{'='*50}")
        
        # Load data
        modis_data, aws_data = data_loader.load_glacier_data(glacier_id)
        
        # Apply pixel selection
        selected_modis = pixel_selector.select_best_pixels(modis_data, glacier_id)
        
        # Process and merge data
        processed = data_processor.merge_and_process(selected_modis, aws_data, glacier_id)
        
        if not processed.empty:
            all_processed_data.append(processed)
            processing_summary.append({
                'glacier': glacier_id,
                'status': 'SUCCESS',
                'methods_processed': len(processed),
                'total_modis_obs': len(modis_data),
                'selected_modis_obs': len(selected_modis),
                'aws_obs': len(aws_data)
            })
            print(f"✓ Successfully processed {glacier_id}: {len(processed)} methods")
        else:
            processing_summary.append({
                'glacier': glacier_id,
                'status': 'FAILED',
                'methods_processed': 0,
                'error': 'No processed data'
            })
            print(f"✗ No processed data for {glacier_id}")
            
    except Exception as e:
        processing_summary.append({
            'glacier': glacier_id,
            'status': 'ERROR',
            'methods_processed': 0,
            'error': str(e)
        })
        print(f"✗ Error processing {glacier_id}: {e}")
        continue

# Display processing summary
print(f"\n{'='*50}")
print("PROCESSING SUMMARY")
print(f"{'='*50}")
summary_df = pd.DataFrame(processing_summary)
print(summary_df.to_string(index=False))

successful_glaciers = len([s for s in processing_summary if s['status'] == 'SUCCESS'])
print(f"\n✓ Successfully processed {successful_glaciers} out of 3 glaciers")

## 7. Generate the Scatterplot Matrix

Now let's create the publication-ready 3×3 scatterplot matrix:

In [ ]:
# Create the scatterplot matrix
if all_processed_data:
    print(f"\n{'='*50}")
    print("Creating Scatterplot Matrix")
    print(f"{'='*50}")
    
    # Generate output filename
    output_path = "aws_vs_modis_scatterplot_matrix.png"
    
    # Create the plot
    fig = visualizer.create_scatterplot_matrix(all_processed_data, output_path)
    
    # Display the plot in notebook
    plt.show()
    
    print(f"\n✓ SUCCESS: Scatterplot matrix generated and saved to {output_path}")
    print(f"✓ Total glaciers in visualization: {len(all_processed_data)}")
    
else:
    print("\n✗ ERROR: No data could be processed for any glacier")
    print("Please check the file paths in the configuration section above.")

## 8. Detailed Results Analysis

Let's examine the detailed statistics for each glacier-method combination:

In [ ]:
# Create detailed results table
if all_processed_data:
    print("\n" + "="*80)
    print("DETAILED STATISTICAL RESULTS")
    print("="*80)
    
    # Combine all processed data
    all_stats = []
    for df in all_processed_data:
        for _, row in df.iterrows():
            all_stats.append({
                'Glacier': row['glacier_id'].title(),
                'Method': row['method'],
                'n': int(row['n_samples']),
                'R': f"{row['correlation']:.3f}",
                'R²': f"{row['r_squared']:.3f}",
                'RMSE': f"{row['rmse']:.3f}",
                'MAE': f"{row['mae']:.3f}",
                'Bias': f"{row['bias']:.3f}",
                'p-value': f"{row['p_value']:.4f}"
            })
    
    results_df = pd.DataFrame(all_stats)
    print(results_df.to_string(index=False))
    
    # Find best performing combinations
    print("\n" + "="*50)
    print("BEST PERFORMING COMBINATIONS")
    print("="*50)
    
    # Convert back to numeric for analysis
    numeric_df = pd.DataFrame(all_stats)
    numeric_df['R_num'] = numeric_df['R'].astype(float)
    numeric_df['RMSE_num'] = numeric_df['RMSE'].astype(float)
    
    # Best correlation
    best_r = numeric_df.loc[numeric_df['R_num'].idxmax()]
    print(f"Highest Correlation: {best_r['Glacier']} - {best_r['Method']} (R = {best_r['R']})")
    
    # Best RMSE (lowest)
    best_rmse = numeric_df.loc[numeric_df['RMSE_num'].idxmin()]
    print(f"Lowest RMSE: {best_rmse['Glacier']} - {best_rmse['Method']} (RMSE = {best_rmse['RMSE']})")
    
    # Summary by glacier
    print("\n" + "="*50)
    print("SUMMARY BY GLACIER")
    print("="*50)
    
    for glacier in ['Athabasca', 'Haig', 'Coropuna']:
        glacier_data = numeric_df[numeric_df['Glacier'] == glacier]
        if not glacier_data.empty:
            avg_r = glacier_data['R_num'].mean()
            avg_rmse = glacier_data['RMSE_num'].mean()
            total_n = glacier_data['n'].sum()
            print(f"{glacier}: Avg R = {avg_r:.3f}, Avg RMSE = {avg_rmse:.3f}, Total n = {total_n}")
    
else:
    print("No results to analyze.")

## 9. Configuration Tips

### Modifying File Paths
If your data files are in different locations, update the `CONFIG['data_paths']` section at the top of this notebook.

### Adjusting Parameters
- **Outlier threshold**: Modify `CONFIG['outlier_threshold']` (default: 2.5σ)
- **Quality filters**: Adjust `CONFIG['quality_filters']['min_glacier_fraction']` and `min_observations`
- **Colors**: Change glacier colors in `CONFIG['colors']`
- **Figure size**: Modify `CONFIG['visualization']['figsize']`

### Adding New Glaciers
To add a new glacier:
1. Add data paths to `CONFIG['data_paths']`
2. Add AWS station coordinates to `CONFIG['aws_stations']`
3. Add color to `CONFIG['colors']`
4. Update the glacier list in the processing loops

### Troubleshooting
- **File not found errors**: Check that all file paths in CONFIG are correct
- **No data processed**: Verify data file formats match expected structure
- **Missing methods**: Check that your MODIS data contains the expected method columns
- **Empty plots**: Ensure adequate data overlap between AWS and MODIS observations